#### Данная тетрадка - предсказания на бейзлайне, т.е. преобразования минимальны, сокращение значений в признаках utm_*, device_* чтобы небыло много ohe-признаков

In [1]:
from warnings import filterwarnings
from datetime import datetime as dt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
import pickle
import pandas as pd
import numpy as np
import gc
from datetime import datetime as dt
filterwarnings('ignore')

In [2]:
# %%time
# %run final_projects_merging.ipynb

In [3]:
datatypes = {'session_id': 'string',
 'client_id': 'string',
 'visit_date': 'string',
 'visit_time': 'string',
 'visit_number': 'int64',
 'utm_source': 'string',
 'utm_medium':'string',
 'utm_campaign':'string',
 'utm_adcontent':'string',
 'utm_keyword':'string',
 'device_category':'string',
 'device_os':'string',
 'device_brand':'string',
 'device_model':'string',
 'device_screen_resolution': 'string',
 'device_browser':'string',
 'geo_country':'string',
 'geo_city':'string',
 'target_event': 'int64'}

df = pd.read_csv("D:\\Libraries\\Projects\\Skillbox.FinalProject\\data\\stage#1.csv", index_col=False, dtype=datatypes )
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

In [4]:
df = df.sample(n=10000).reset_index(drop=True)

for col in df.columns:
    df[col] = df[col].fillna('unknown')
    
col_for_del = ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number']
df = df.drop(col_for_del, axis=1)

cols_for_ohe = ['device_category', 'utm_source', 'utm_campaign', 'utm_medium', 'utm_adcontent', 'utm_keyword', 'device_brand', 'device_screen_resolution', 
                'device_os', 'device_model', 'device_browser', 'geo_country', 'geo_city']

ohe = OneHotEncoder(sparse=False)


ohe.fit(df[cols_for_ohe])
transformed = ohe.transform(df[cols_for_ohe])
df[ohe.get_feature_names_out()] = transformed
df_ohed = df.drop(cols_for_ohe, axis=1)

In [5]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# df_ohed = df_ohed.sample(n=50000).reset_index(drop=True)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, auc
x = df_ohed.drop(['target_event'], axis=1)
y = df_ohed['target_event']
del df_ohed
gc.collect()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

#=======================================================================================================================
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(class_weight='balanced', random_state=42)
# clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(x_train, y_train)

proba_test = clf.predict_proba(x_test)
proba_test = proba_test[:, 1]
proba_train = clf.predict_proba(x_train)
proba_train = proba_train[:, 1]

predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predicted_train)
score_test  = accuracy_score(y_test, predicted_test)
print('DesicionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predicted_test))
# del clf
# gc.collect()
#=====================================================================
print(f'RandomForestClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1, random_state=42)
forest.fit(x_train, y_train)

proba_test = forest.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest.predict(x_train)
predict_test  = forest.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nRandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))
# del forest
# gc.collect()
#=====================================================================
print(f'\nExtraTreeClassifier:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
from sklearn.ensemble import ExtraTreesClassifier

extree = ExtraTreesClassifier(class_weight='balanced', random_state=42)
extree.fit(x_train, y_train)

proba_test = extree.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree.predict_proba(x_train)
proba_train = proba_train[:, 1]

predict_train = extree.predict(x_train)
predict_test  = extree.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
# print('\nExtraTreesClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

print(f'End cell:   {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')

Start cell: 2024-03-25 10:47:03
DesicionTreeClassifier:
 Качество на тренировочной выборке: 0.9624285714285714
 Качество на тестовой выборке:      0.9106666666666666
 ROc AUC на тренировочной выборке:  0.9965683429932439
 ROC AUC на тестовой выборке:       0.53191101217417
[[2723  203]
 [  65    9]]
RandomForestClassifier:   2024-03-25 10:47:03
 Качество на тренировочной выборке: 0.9682857142857143
 Качество на тестовой выборке:      0.9433333333333334
 ROc AUC на тренировочной выборке:  0.9880309552163905
 ROC AUC на тестовой выборке:       0.6073068112541796
[[2827   99]
 [  71    3]]

ExtraTreeClassifier:   2024-03-25 10:47:11
 Качество на тренировочной выборке: 0.9624285714285714
 Качество на тестовой выборке:      0.9373333333333334
 ROc AUC на тренировочной выборке:  0.9965683429932439
 ROC AUC на тестовой выборке:       0.5685166540429699
[[2809  117]
 [  71    3]]
End cell:   2024-03-25 10:47:17
CPU times: total: 1min 13s
Wall time: 14.4 s


In [6]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params_dtc = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


dtc = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=dtc,
    param_grid=params_dtc,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

dtc_tuned = DecisionTreeClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion'],
    splitter=         best_params['splitter']
)

dtc_tuned.fit(x_train, y_train)
proba_test = dtc_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = dtc_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = dtc_tuned.predict(x_train)
predict_test  = dtc_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('DecisionTreeClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 10:47:17
Fitting 3 folds for each of 4140 candidates, totalling 12420 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 14, 'max_features': None, 'min_samples_leaf': 34, 'splitter': 'best'}
DecisionTreeClassifier:
 Качество на тренировочной выборке: 0.8112857142857143
 Качество на тестовой выборке:      0.811
 ROc AUC на тренировочной выборке:  0.7498545381381969
 ROC AUC на тестовой выборке:       0.6249861447229867
[[2414  512]
 [  55   19]]
CPU times: total: 1min 6s
Wall time: 4min 34s


In [7]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')


params_forest = {
    'class_weight': ['balanced'],
    'n_estimators': list(range(25, 45)),
    'max_depth': list(range(5, 10)),
    'min_samples_leaf': list(range(12, 25)),
    'max_features': ['sqrt', 'log2', None]
}


forest = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=forest,
    param_grid=params_forest,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

forest_tuned = RandomForestClassifier(
    class_weight=     'balanced',
    n_estimators=     best_params['n_estimators'],
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features']
)

forest_tuned.fit(x_train, y_train)
proba_test = forest_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = forest_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]


predict_train = forest_tuned.predict(x_train)
predict_test  = forest_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('RandomForestClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 10:51:51
Fitting 3 folds for each of 3900 candidates, totalling 11700 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 12, 'n_estimators': 25}
RandomForestClassifier:
 Качество на тренировочной выборке: 0.619
 Качество на тестовой выборке:      0.6283333333333333
 ROc AUC на тренировочной выборке:  0.7003178798069416
 ROC AUC на тестовой выборке:       0.6393609946241525
[[1844 1082]
 [  33   41]]
CPU times: total: 42.4 s
Wall time: 26min


In [8]:
%%time
print(f'Start cell: {dt.now().strftime("%Y-%m-%d %H:%M:%S")}')
# from sklearn.model_selection import GridSearchCV

params_extree = {
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy', 'log_loss'],
#     'splitter': ['best', 'randoms'],
    'max_depth': list(range(5, 15)),
    'min_samples_leaf': list(range(12, 35)),
    'max_features': ['sqrt', 'log2', None]
}


extree = ExtraTreesClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=extree,
    param_grid=params_extree,
    cv=3,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print(f'Лучшие параметры:\n\t{best_params}')

extree_tuned = ExtraTreesClassifier(
    class_weight=     'balanced',
    max_depth=        best_params['max_depth'],
    min_samples_leaf= best_params['min_samples_leaf'],
    max_features=     best_params['max_features'],
    criterion=        best_params['criterion']
#     splitter=         best_params['splitter']
)

extree_tuned.fit(x_train, y_train)
proba_test = extree_tuned.predict_proba(x_test)
proba_test = proba_test[:,1]
proba_train = extree_tuned.predict_proba(x_train)
proba_train = proba_train[:, 1]

    
predict_train = extree_tuned.predict(x_train)
predict_test  = extree_tuned.predict(x_test)

proba_test_y = roc_auc_score(y_test, proba_test)
proba_train_y = roc_auc_score(y_train, proba_train)
score_train = accuracy_score(y_train, predict_train)
score_test  = accuracy_score(y_test,  predict_test)
print('ExtraTreesClassifier:')
print(' Качество на тренировочной выборке:', score_train)
print(' Качество на тестовой выборке:     ', score_test)
print(' ROc AUC на тренировочной выборке: ', proba_train_y)
print(' ROC AUC на тестовой выборке:      ', proba_test_y)
print(confusion_matrix(y_test, predict_test))

Start cell: 2024-03-25 11:17:51
Fitting 3 folds for each of 2070 candidates, totalling 6210 fits
Лучшие параметры:
	{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 19}
ExtraTreesClassifier:
 Качество на тренировочной выборке: 0.6661428571428571
 Качество на тестовой выборке:      0.6766666666666666
 ROc AUC на тренировочной выборке:  0.7655193629353078
 ROC AUC на тестовой выборке:       0.6257574218100534
[[1998  928]
 [  42   32]]
CPU times: total: 23.4 s
Wall time: 57min 52s
